In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
dir0 = os.path.dirname(dir1)
if not dir1 in sys.path: sys.path.append(dir0)

In [21]:
from brio.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from functools import reduce
from itertools import compress

In [3]:
from brio.bias.FreqVsFreqBiasDetector import FreqVsFreqBiasDetector, BiasDetector
from brio.risk.HazardFromBiasDetectionCalculator import HazardFromBiasDetectionCalculator
from brio.risk.RiskCalculator import RiskCalculator

In [4]:
model_1 = pd.read_csv("./data/processed_model01.csv")
model_2 = pd.read_csv("./data/processed_model02.csv")
model_3 = pd.read_csv("./data/processed_model03.csv")
model_4 = pd.read_csv("./data/processed_model04.csv")
model_5 = pd.read_csv("./data/processed_model05.csv")

datasets = [model_1, model_2, model_3, model_4, model_5]
names = ["model_1", "model_2", "model_3", "model_4", "model_5"]

datasets_dict = {
    "model_1": model_1,
    "model_2": model_2,
    "model_3": model_3,
    "model_4": model_4,
    "model_5": model_5
}

In [14]:
sensitive_list = ['foreign_worker', 'personal_status_and_sex', 'age_group']
threshold=0.15
mode = 'group'
distance="TVD"
A1='high'
min_obs_per_group=30
aggregating_function=max
business_list_by_model = {"model_1": ['status', 'duration', 'credit_history'],
                          "model_2": ['status', 'duration', 'credit_history'],
                          "model_3": ['status', 'property', 'credit_history'],
                          "model_4": ['status', 'duration', 'credit_history'],
                          "model_5": ['credit_history', 'number_of_credits', 'employment_duration']                         }

## Risk Path con valori percentuali

In [73]:
path = {"step_1": [False, False, False],
        "step_2": [True, False, False],
        "step_3": [True, True, False],
        "step_4": [True, True, True]}

In [74]:
results = {}
for model in names:
    print(model)
    
    results[model] = {}

    conditioning_variables = business_list_by_model[model]
    print(conditioning_variables)
    
    for path_name, path_rules in path.items():

        path_variables = list(compress(conditioning_variables, path_rules))
        
        model_hazards_for_a_path = []
        for sensitive in sensitive_list:
            bias = bd.compare_root_variable_groups(
                    dataframe=datasets_dict[model],
                    target_variable='predicted_label',
                    root_variable=sensitive,
                    threshold=threshold
                )  

            if len(path_variables) > 0:
                bias_conditioned = bd.compare_root_variable_conditioned_groups(
                        dataframe=datasets_dict[model],
                        target_variable='predicted_label',
                        conditioning_variables=path_variables,
                        root_variable=sensitive,
                        threshold=threshold,
                        min_obs_per_group=min_obs_per_group
                    )
            else:
                bias_conditioned = None


            hazard = hc.compute_hazard_from_freqvsfreq_or_freqvsref(
                    overall_result=bias,
                    conditioned_results=bias_conditioned,
                    tot_observations=datasets_dict[model].shape[0],
                    conditioning_variables=business_list_by_model[model],
                    weight_logic=mode
                )
            
            model_hazards_for_a_path.append(hazard[0]/hazard[-1]) #just take the overall hazard and divides by max
            
        path_results_for_a_model = np.mean(model_hazards_for_a_path)
            
        results[model][path_name] = path_results_for_a_model   

model_1
['status', 'duration', 'credit_history']
model_2
['status', 'duration', 'credit_history']
model_3
['status', 'property', 'credit_history']
model_4
['status', 'duration', 'credit_history']
model_5
['credit_history', 'number_of_credits', 'employment_duration']


In [75]:
pd.DataFrame(results)

,model_1,model_2,model_3,model_4,model_5
step_1,1.000000,1.000000,0.666667,0.666667,0.333333
step_2,0.700924,0.700924,0.673632,0.586305,0.315642
step_3,0.771843,0.771843,0.778331,0.628304,0.414151
step_4,0.758811,0.758811,0.779442,0.642159,0.595499


In [76]:
pd.DataFrame(results).to_clipboard()

## Risk Path con valori assoluti

In [67]:
path = {"step_1": [False, False, False],
        "step_2": [True, False, False],
        "step_3": [True, True, True],
        "step_4": [True, True, True]}

In [68]:
results = {}
for model in names:
    print(model)
    
    results[model] = {}

    conditioning_variables = business_list_by_model[model]
    print(conditioning_variables)
    
    for path_name, path_rules in path.items():

        path_variables = list(compress(conditioning_variables, path_rules))
        
        model_hazards_for_a_path = []
        for sensitive in sensitive_list:
            bias = bd.compare_root_variable_groups(
                    dataframe=datasets_dict[model],
                    target_variable='predicted_label',
                    root_variable=sensitive,
                    threshold=threshold
                )  

            if len(path_variables) > 0:
                bias_conditioned = bd.compare_root_variable_conditioned_groups(
                        dataframe=datasets_dict[model],
                        target_variable='predicted_label',
                        conditioning_variables=path_variables,
                        root_variable=sensitive,
                        threshold=threshold,
                        min_obs_per_group=min_obs_per_group
                    )
            else:
                bias_conditioned = None


            hazard = hc.compute_hazard_from_freqvsfreq_or_freqvsref(
                    overall_result=bias,
                    conditioned_results=bias_conditioned,
                    tot_observations=datasets_dict[model].shape[0],
                    conditioning_variables=business_list_by_model[model],
                    weight_logic=mode
                )
            
            model_hazards_for_a_path.append(hazard[0]) #just take the overall hazard and divides by max
            
        path_results_for_a_model = np.mean(model_hazards_for_a_path)
            
        results[model][path_name] = path_results_for_a_model   

model_1
['status', 'duration', 'credit_history']
model_2
['status', 'duration', 'credit_history']
model_3
['status', 'property', 'credit_history']
model_4
['status', 'duration', 'credit_history']
model_5
['credit_history', 'number_of_credits', 'employment_duration']


In [69]:
pd.DataFrame(results)

,model_1,model_2,model_3,model_4,model_5
step_1,0.150128,0.150128,0.149815,0.124754,0.018469
step_2,0.079050,0.079050,0.079604,0.070225,0.030855
step_3,0.062602,0.062602,0.059967,0.054042,0.042772
step_4,0.062602,0.062602,0.059967,0.054042,0.042772


## Threshold analysis

In [82]:
results = {}
for model in names:
    print(model)
    
    thresholds = np.arange(0.05, 1, 0.05)
    
    calibration_results_for_a_model = []
    for t in thresholds:
        print("Threshold: ", t)

        path_variables = list(compress(conditioning_variables, path_rules))
        
        model_hazards_for_a_threshold = []
        for sensitive in sensitive_list:
            bias = bd.compare_root_variable_groups(
                    dataframe=datasets_dict[model],
                    target_variable='predicted_label',
                    root_variable=sensitive,
                    threshold=t
                )  

            bias_conditioned = bd.compare_root_variable_conditioned_groups(
                    dataframe=datasets_dict[model],
                    target_variable='predicted_label',
                    conditioning_variables=business_list_by_model[model],
                    root_variable=sensitive,
                    threshold=t,
                    min_obs_per_group=min_obs_per_group
                )


            hazard = hc.compute_hazard_from_freqvsfreq_or_freqvsref(
                    overall_result=bias,
                    conditioned_results=bias_conditioned,
                    tot_observations=datasets_dict[model].shape[0],
                    conditioning_variables=business_list_by_model[model],
                    weight_logic=mode
                )
            
            model_hazards_for_a_threshold.append(hazard[0]/hazard[-1]) #just take the overall hazard and divides by max

        calibration_results_for_a_model.append(np.nanmean(model_hazards_for_a_threshold))
            
    results[model] = calibration_results_for_a_model   

model_1
Threshold:  0.05
Threshold:  0.1
Threshold:  0.15000000000000002
Threshold:  0.2
Threshold:  0.25
Threshold:  0.3
Threshold:  0.35000000000000003
Threshold:  0.4
Threshold:  0.45
Threshold:  0.5
Threshold:  0.55
Threshold:  0.6000000000000001
Threshold:  0.6500000000000001
Threshold:  0.7000000000000001
Threshold:  0.7500000000000001
Threshold:  0.8
Threshold:  0.8500000000000001
Threshold:  0.9000000000000001
Threshold:  0.9500000000000001
model_2
Threshold:  0.05
Threshold:  0.1
Threshold:  0.15000000000000002
Threshold:  0.2
Threshold:  0.25
Threshold:  0.3
Threshold:  0.35000000000000003
Threshold:  0.4
Threshold:  0.45
Threshold:  0.5
Threshold:  0.55
Threshold:  0.6000000000000001
Threshold:  0.6500000000000001
Threshold:  0.7000000000000001
Threshold:  0.7500000000000001
Threshold:  0.8
Threshold:  0.8500000000000001
Threshold:  0.9000000000000001
Threshold:  0.9500000000000001
model_3
Threshold:  0.05
Threshold:  0.1
Threshold:  0.15000000000000002
Threshold:  0.2
Thres

In [111]:
pd.DataFrame(results)

,model_1,model_2,model_3,model_4,model_5
0,0.910790,0.910790,0.982148,0.919957,0.873979
1,0.855717,0.855717,0.940810,0.849463,0.765397
2,0.758811,0.758811,0.779442,0.642159,0.595499
3,0.541348,0.541348,0.597425,0.500602,0.394506
4,0.463862,0.463862,0.482326,0.360566,0.282226
5,0.384947,0.384947,0.346804,0.248202,0.217867
6,0.259562,0.259562,0.252995,0.157996,0.177320
7,0.179602,0.179602,0.200446,0.133942,0.150794
8,0.137627,0.137627,0.164452,0.116313,0.114965
9,0.104409,0.104409,0.131690,0.093944,0.096114


In [112]:
pd.DataFrame(results).to_clipboard()

In [115]:
pd.Series(thresholds).to_clipboard()